In [ ]:
import requests
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import time

In [ ]:
standings_url = "https://fbref.com/en/comps/12/2019-2020/schedule/2019-2020-La-Liga-Scores-and-Fixtures"
data = requests.get(standings_url)
data.text

In [ ]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [ ]:
matches

In [ ]:
matches.to_csv("matches_2019-2020.csv")

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'en/matches/' in l]

In [ ]:
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
data.text

In [ ]:
match_report = pd.read_html(data.text, match="Match Report")[0]

In [ ]:
possession.columns = possession.columns.droplevel()

In [ ]:
team_data1 = matches.merge(possession[["Date", "Touches", "Att 3rd", "Att Pen", "TotDist", "PrgDist", "PrgC", "CPA", "Rec", "PrgR"]], on="Date")

In [ ]:
years = list(range(2024, 2020, -1))
all_matches = []
years

In [ ]:
standings_url = "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"

In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/possession/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        possession = pd.read_html(data.text, match="Possession")[0]
        possession.columns = possession.columns.droplevel()
        try:
            team_data = matches.merge(possession[["Date", "Touches", "Att 3rd", "Att Pen", "TotDist", "PrgDist", "PrgC", "CPA", "Rec", "PrgR"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(20)

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches_possession.csv")